In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans
from tensorflow.examples.tutorials.mnist import input_data


# 加载数据集
mnist = input_data.read_data_sets('./data/fashion', one_hot=True)
train_image = mnist.train.images

num_steps = 50  # 训练次数
k = 25
num_classes = 10

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, num_classes])

kmeans = KMeans(inputs=x, num_clusters=k, distance_metric='cosine', use_mini_batch=True)

(all_scores, cluster_idx, scores, cluster_centers_initialized, init_op, train_op) = kmeans.training_graph()

cluster_idx = cluster_idx[0]  # 转换为元组
avg_distance = tf.reduce_mean(scores)  # 求均值

# 开启会话
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(init_op, feed_dict={x: train_image})

    # 训练
    for i in range(1, num_steps+1):
        _, d, idx = sess.run([train_op, avg_distance, cluster_idx], feed_dict={x: train_image})
        # print(_, d, idx)
        # print('*'*20)
        print('Step %i, 平均距离：%f' % (i, d))

    # 计算每个质心的标签总数，使用每个训练的标签。
    # 由编号找到距离最近的质心
    counts = np.zeros(shape=(k, num_classes))

    for i in range(len(idx)):
        counts[idx[i]] += mnist.train.labels[i]

    labels_map = tf.convert_to_tensor([np.argmax(x) for x in counts])

    cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)

    # 计算精度
    correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(y, 1), tf.int32))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 测试模型
    test_x, test_y = mnist.test.next_batch(10000)
    print("Test Accuracy:", sess.run(accuracy_op, feed_dict={x: test_x, y: test_y}))

writer.close()

Extracting ./data/fashion/train-images-idx3-ubyte.gz
Extracting ./data/fashion/train-labels-idx1-ubyte.gz
Extracting ./data/fashion/t10k-images-idx3-ubyte.gz
Extracting ./data/fashion/t10k-labels-idx1-ubyte.gz
Step 1, 平均距离：0.160441
Step 2, 平均距离：0.106566
Step 3, 平均距离：0.103422
Step 4, 平均距离：0.102370
Step 5, 平均距离：0.101845
Step 6, 平均距离：0.101509
Step 7, 平均距离：0.101265
Step 8, 平均距离：0.101069
Step 9, 平均距离：0.100906
Step 10, 平均距离：0.100762
Step 11, 平均距离：0.100627
Step 12, 平均距离：0.100493
Step 13, 平均距离：0.100356
Step 14, 平均距离：0.100223
Step 15, 平均距离：0.100096
Step 16, 平均距离：0.099973
Step 17, 平均距离：0.099858
Step 18, 平均距离：0.099752
Step 19, 平均距离：0.099655
Step 20, 平均距离：0.099565
Step 21, 平均距离：0.099483
Step 22, 平均距离：0.099408
Step 23, 平均距离：0.099340
Step 24, 平均距离：0.099277
Step 25, 平均距离：0.099220
Step 26, 平均距离：0.099166
Step 27, 平均距离：0.099117
Step 28, 平均距离：0.099070
Step 29, 平均距离：0.099027
Step 30, 平均距离：0.098987
Step 31, 平均距离：0.098949
Step 32, 平均距离：0.098913
Step 33, 平均距离：0.098880
Step 34, 平均距离：0.098849
Step 35, 平均距离：0.0

![](./img/k-means.png) 

![](./images/k-means.png)